In [43]:
import torch
from utils import LQR_2D_true_solution, out_put, LQR_1D_true_solution, l_2_compute_1D_Q, l_2_compute_1D_V, true_V_eval_2D
from main_RL import RL_finder_1D_LQR, RL_finder_2D_LQR
from bases import bases_poly, d_bases_poly, sec_bases_poly, bases_2d, bases_poly_simp, d_bases_poly_simp, sec_bases_poly_simp, bases_2d_simp, bases_poly_2D, bases_poly_2D_s_a, d_bases_poly_2D, bases_poly_2D_simp, d_bases_poly_2D_simp, sec_d_bases_poly_2D, sec_d_bases_poly_2D_simp, bases_poly_2D_s_a_simp
import matplotlib.pyplot as plt
from math import sqrt
import numpy as np

In [2]:
torch.set_default_dtype(torch.float64)
device = torch.device("cpu")

In [3]:
# 1D case

In [4]:
def reward(s, a):
    return - Q * (s**2) - R * (a**2)

In [25]:
A = 0.2
B = 0.2
sig = 0.6
R = 1
Q = 1
S = 0
beta = 3
M = 2
info_true = {"A":A, "B":B, "sig":sig, "R":R, "Q":Q,}
true_b, True_V = LQR_1D_true_solution(A, B, Q, R, sig, beta)
print(true_b, True_V)

-0.07647321898295312 tensor([-0.0459,  0.0000, -0.3824])


In [26]:
b_init = - 0.52
Q_init = torch.tensor([1., -1., -1., -1.]).to(device)
batch_size = int(1e6)
bd_low_s = -2
bd_upper_s = 2.
bd_low_b = -2.
bd_upper_b = 3.
bd_low_a = -3.
bd_upper_a = 3
bases_Q = bases_2d
num_iter = 80
GD_num_iter = 80
I = 6
m = int(batch_size / I)
m_Q = int(batch_size / I)
lr = 0.01
dt = 0.001
order = 1
true_V = True_V
simple_basis = False
b_val_samp_1, V_exact_dist_samp_1 = RL_finder_1D_LQR(beta, b_init, bd_low_s, bd_upper_s, bd_low_a, bd_upper_a, reward, simple_basis, bases_Q, num_iter, m_Q, I, dt, true_V, info_true)
V_exact_dist_samp_1

Running RL method: 100%|██████████| 80/80 [00:03<00:00, 22.14it/s]


[0.706160019669420,
 0.00520736428696482,
 0.000439003884240282,
 0.000195129214044501,
 0.00241998156025205,
 0.000823348352860264,
 0.00146724336979905,
 0.00285277871992389,
 1.78829963651638e-5,
 3.01045475589595e-5,
 0.00298767342337984,
 0.000274039377858607,
 0.000794216391222938,
 0.000116345819839128,
 5.45145740602661e-5,
 0.00170625992518903,
 0.000612336992351636,
 0.000178437723668713,
 0.000464011738754122,
 1.06359329517138e-5,
 0.000133569076747074,
 0.000377090028762593,
 0.000981989629706688,
 7.91960308751612e-5,
 0.000340017156879904,
 0.000765387380171593,
 4.08300408755565e-6,
 0.000163793797331895,
 0.00444812103297018,
 3.21593034270621e-5,
 0.000202881269981099,
 0.000865380546503891,
 0.000140320052480786,
 0.000151814260617843,
 7.74625889025144e-7,
 0.000820088383380627,
 0.000869108768440744,
 0.00151671471655117,
 0.00107830002874244,
 0.000704161948267796,
 0.00182101731754149,
 6.36352767855690e-6,
 4.64331814926567e-5,
 6.06608370598362e-6,
 0.000206234

In [24]:
print(b_val_samp_1)

[-0.52, tensor(-0.0904), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor(-0.0764), tensor

In [4]:
# 2D case

In [5]:
def reward(traj_mat, act_mat):
    # traj_mat shape: (m, I, dim)
    # act_mat shape: (m, I, dim)
    # output shape: (m, I, 1)
    r_1 = torch.einsum("ijk,kl,ijl->ij", traj_mat, Q, traj_mat)
    r_2 = torch.einsum("ijk,kl,ijl->ij", act_mat, R, act_mat)
    return (- r_1 - r_2).unsqueeze(-1)

In [38]:
beta = 5
A =  torch.inverse(0.2 * torch.tensor([[-3.0, 1.0], [1., -3.0]]))
B =  0.2 * torch.tensor([[3.0, 1.0], [1., 3.0]])
# C = - 0.1 * torch.tensor([[2.0, 1.0], [1., 1.0]])
# D = 0.1 * torch.tensor([[3.0, 2.0], [.5, 1.0]])
sig = 0
Q = 1 * torch.tensor([[0.54, -0.86], [-0.86, 1.5]])
R = 1 * torch.inverse(torch.tensor([[1.92, 0.4], [0.4, 0.3]]))
# Q = 1 * torch.tensor([[0.54, -0.26], [-0.26, 0.4]])
# R = 1 * torch.inverse(torch.tensor([[1.92, 0.4], [0.4, 0.3]]))
# Q = 1 * torch.tensor([[0.9, 0.89], [0.89, 0.9]])
# R = 1 * torch.inverse(torch.tensor([[.4, 0.398], [0.398, 0.4]]))

In [39]:
A_norm = torch.sqrt(torch.abs(torch.linalg.eig(A.T @ A).eigenvalues).max())
A_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(A).T @ torch.inverse(A)).eigenvalues).max())
B_norm = torch.sqrt(torch.abs(torch.linalg.eig(B.T @ B).eigenvalues).max())
B_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(B).T @ torch.inverse(B)).eigenvalues).max())
print(A_norm)
print(A_inv_norm)
print(B_norm)
print(B_inv_norm)
print(A_norm**2 * A_inv_norm * B_norm * B_inv_norm**2)

tensor(2.5000)
tensor(0.8000)
tensor(0.8000)
tensor(2.5000)
tensor(25.0000)


In [40]:
Q_norm = torch.sqrt(torch.abs(torch.linalg.eig(Q.T @ Q).eigenvalues).max())
Q_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(Q).T @ torch.inverse(Q)).eigenvalues).max())
R_norm = torch.sqrt(torch.abs(torch.linalg.eig(R.T @ R).eigenvalues).max())
R_inv_norm = torch.sqrt(torch.abs(torch.linalg.eig(torch.inverse(R).T @ torch.inverse(R)).eigenvalues).max())
print(Q_norm)
print(Q_inv_norm)
print(R_norm)
print(R_inv_norm)
print(Q_norm**2 * Q_inv_norm * R_norm * R_inv_norm**2)

tensor(2.0049)
tensor(28.4785)
tensor(4.8399)
tensor(2.0134)
tensor(2245.8555)


In [41]:
true_b, true_c, True_V = LQR_2D_true_solution(A, B, sig, Q, R, beta)
b_init = true_b + torch.randn(2, 2)
c_init = 2. * torch.rand(2, 1) - 1.
Q_init = 4. * torch.rand(10) - 2.
V_init = true_V_eval_2D(A, B, b_init, R, Q, beta, sig)

In [42]:
print(true_b)
print(true_c)
print(True_V)
print(V_init)

tensor([[-0.0234,  0.0271],
        [ 0.0070, -0.0137]])
tensor([[0.],
        [0.]])
tensor([ 0.0000, -0.1879,  0.2343, -0.0783])
tensor([ 0.0000, -0.4098,  1.4119, -1.8218])


In [44]:
bd_low_s = -2
bd_upper_s = 2
bd_low_a = -2
bd_upper_a = 2
reward = reward
bases_Q = bases_poly_2D_s_a_simp
num_iter = 20
GD_num_iter = 75
m=int(1e6)
m_Q=int(1e6)
I=6
I_Q=6
true_V = True_V
info_true = {"A":A, "B":B, "sig":sig, "R":R, "Q":Q}
dt = 0.001
simple_basis = True
b_val_GD, V_exact_dist_1 = RL_finder_2D_LQR(beta, b_init, bd_low_s, bd_upper_s, bd_low_a, bd_upper_a, reward, simple_basis, bases_Q, num_iter, I, m_Q, dt, true_V, info_true)

Running RL method: 100%|██████████| 20/20 [00:14<00:00,  1.42it/s]


In [45]:
V_exact_dist_1

[50.0132183143718,
 4.824221235823765,
 0.04992104431614899,
 4.248204445542779e-06,
 1.0763719685405871e-07,
 1.0770364565360164e-07,
 1.0770364595293298e-07,
 1.0770364533190083e-07,
 1.0770364976177951e-07,
 1.0770364535427028e-07,
 1.0770364565360164e-07,
 1.0770364565360164e-07,
 1.0770364563123217e-07,
 1.0770364876897709e-07,
 1.0770364565360164e-07,
 1.0770364876897709e-07,
 1.0770364535427028e-07,
 1.0770364565360164e-07,
 1.0770364535427028e-07,
 1.0770364535427028e-07]

In [46]:
b_val_GD

[tensor([[ 0.4920,  0.4707],
         [-1.5929,  0.6245]]),
 tensor([[-1.7957,  0.6119],
         [-0.3613,  0.1049]]),
 tensor([[-0.1830,  0.0885],
         [-0.0252, -0.0014]]),
 tensor([[-0.0249,  0.0276],
         [ 0.0066, -0.0135]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 0.0069, -0.0137]]),
 tensor([[-0.0233,  0.0270],
         [ 